# CUDA Quantum 101
    Important Links 
    
    * Installation (Docker recommended)
        https://nvidia.github.io/cuda-quantum/latest/install.html 
    * Documentation
        https://nvidia.github.io/cuda-quantum/latest/index.html
    * CUDA Quantum repo
        https://github.com/NVIDIA/cuda-quantum


    Outline 

    1. What is CUDA Quantum? 
    2. CUDA Quantum Kernels
    3. CUDA Q Algorithmic Primitives
        3.1 cudaq.sample() 
        3.2 cudaq.spin_op()
        3.3 cudaq.observe()
    4. Parameterized circuits 
    5. Noise-modeling

## Excerpts from a 2018 DOE report
<img src="excerpts_report2018.png" alt="Image Title" width="700">

In [4]:
# Import the CUDA Quantum module
import cudaq

### 2. CUDA Quantum Kernel 

# We begin by defining the `Kernel` that we will construct our
# program with.
kernel = cudaq.make_kernel()

In [ ]:
# Next, we can allocate qubits to the kernel via `qalloc(qubit_count)`.
# An empty call to `qalloc` will return a single qubit.
qubit = kernel.qalloc()

In [ ]:
# Now we can begin adding instructions to apply to this qubit!
# Here we'll just add every non-parameterized
# single qubit gates that are supported by CUDA Quantum.
kernel.h(qubit)
kernel.x(qubit)
kernel.y(qubit)
kernel.z(qubit)
kernel.t(qubit)
kernel.s(qubit)

In [ ]:
# Next, we add a measurement to the kernel so that we can sample
# the measurement results on our simulator!
kernel.mz(qubit)

In [ ]:
# Other methods and attributes available to the kernel object
#dir(kernel)
#help(kernel.tdg)

1. CUDA Quantum Kernels continued
        1.1 Adjoint 
        1.2 Conditionals 

###     3. Algorithmic primitives

In [ ]:
  Algorithmic primitives are common programming patterns that have been implemented in the CUDA Quantum library.

    3.1 cudaq.sample()
    3.2 cudaq.observe()
    3.3 cudaq.spin_op()

#### 3.1. cudaq.sample()

   The sample() function performs multiple measurements of the circuit(1000 shots by default) and returns a dictionary of the measurement outcomes along with their respective counts. 

In [ ]:
# Finally, we can execute this kernel on the state vector simulator
# by calling `cudaq.sample`. This will execute the provided kernel
# `shots_count` number of times and return the sampled distribution
# as a `cudaq.SampleResult` dictionary.
sample_result = cudaq.sample(kernel)

# Now let's take a look at the `SampleResult` we've gotten back!
print(sample_result)  # or result.dump()     

    Putting it all together!

In [ ]:
import cudaq

kernel = cudaq.make_kernel()
qubit = kernel.qalloc(2)
                                  
kernel.h(qubit)
kernel.x(qubit)
kernel.y(qubit)
kernel.z(qubit)
kernel.t(qubit)
kernel.s(qubit)

kernel.mz(qubit)

# 1000 is the default
sample_result = cudaq.sample(kernel, shots_count=2000) 

print(sample_result)  # or sample_result.dump() 

In [ ]:
# Other methods and attributes available to the kernel object
#dir(sample_result)
#help(sample_result.values)

In [ ]:
# Extracting data from sample

print(f"most probable = {sample_result.most_probable()}")
print(f"expectation_value = {sample_result.expectation_z()}")
print(f"count = {sample_result.count('1')}")
print(f"probability = {sample_result.probability('1')}")


In [ ]:
# clear results, result should now be empty
sample_result.clear()

####  3.2. cudaq.spin_op()

     
    The spin_op represents a general sum of Pauli tensor products. It exposes the typical algebraic operations that allow programmers to define primitive Pauli operators and use them to compose larger, more complex Pauli tensor products and sums thereof. 

Let's say our Hamitonian is $Z_0 \otimes I_1 + I_0 \otimes X_1 + Y_0 \otimes I_1 + Y_0 \otimes Y_1$.

In [ ]:
# Importing the spin_op
from cudaq import spin

# the obseravle 
hamiltonian = spin.z(0) + spin.x(1) + spin.y(0) + spin.y(0)*spin.y(1)
hamiltonian.dump()

In [ ]:
#dir(hamiltonian)
#help(hamiltonian.dump)

#### 3.3. cudaq.observe()

Compute the expected value of the observable.

In [ ]:
# First we need to construct a cuda quantum kernel
kernel = cudaq.make_kernel()
qreg = kernel.qalloc(2)
kernel.x(qreg[0])

In [ ]:
# The cudaq.observe() takes the quantum circuit and the observable as input params
observe_result = cudaq.observe(kernel, hamiltonian, shots_count=1000)

In [ ]:
print(observe_result.dump())
observe_result.expectation_z()

In [ ]:
# For a complete list of attributes
# dir (observe_result)

    Putting it all together!

In [ ]:
import cudaq
from cudaq import spin

# First we need to construct a cuda quantum kernel
kernel = cudaq.make_kernel()
qreg = kernel.qalloc(2)
kernel.x(qreg[0])

# The cudaq.observe() takes the quantum circuit and the observable as input params
observe_result = cudaq.observe(kernel, hamiltonian, shots_count=10000)

print(observe_result.dump())
observe_result.expectation_z()

### 4. Parameterized circuits

In [ ]:
import cudaq
from cudaq import spin

# the obserable 
hamiltonian = 5.907 - 2.1433 * spin.x(0) * spin.x(1) \
            - 2.1433 * spin.y(0) * spin.y(1) + 0.21829 * spin.z(0) \
            - 6.125 * spin.z(1)

# parameterized cudaq kernel, the parameter is of type float
kernel, theta = cudaq.make_kernel(float)
q = kernel.qalloc(2)
kernel.x(q[0])
kernel.ry(theta, q[1])
kernel.cx(q[1], q[0])

# observe() takes the kernel, the observable and the kernel paramter(s)
# as args
observe_result = cudaq.observe(kernel, hamiltonian, .59)
print(observe_result.dump())
observe_result.expectation_z()

#### 3.1 Quantum Hardware Integration

In [1]:
# This code will give an error!!!!!
import cudaq

# Set the target 
cudaq.set_target("quantinuum")

# Create the kernel we'd like to execute on Quantinuum.
kernel = cudaq.make_kernel()
qubits = kernel.qalloc(2)
kernel.h(qubits[0])
kernel.cx(qubits[0], qubits[1])
kernel.mz(qubits)

# Submit to Quantinuum's endpoint and confirm the program is valid.

# By using the synchronous `cudaq.sample`, the execution of
# any remaining classical code in the file will occur only
# after the job has been executed by the Quantinuum service.
# We will use the synchronous call to submit to the syntax
# checker to confirm the validity of the program.
counts = cudaq.sample(kernel)
counts.dump()
assert (len(counts) == 2)
assert ('00' in counts)
assert ('11' in counts)

RuntimeError: Cannot find Quantinuum Config file with credentials (~/.quantinuum_config).

### 4. Noise modeling

Common sources of errors in quantum computation.

    1. Quantum gate errors
    2. Measurement errors
    3. Decoherence
    4. Crosstalk

    * Decoherence refers to the process by which a quantum system loses its quantum coherence, or its ability to exist in a superposition of states due to interaction with the environment.
    
    The evolution of a quantum system that interacts with its environment is described using quantum channels. 

##### Kraus Representation

    The different sources of noise that we discussed above can be represnted mathematically using the Kraus operators.
    
\begin{equation*}
\rho \mapsto {\cal{N}}(\rho) = \sum_{j} K_j \rho K_j^{\dag}
\end{equation*}

    with the condition that 
    
\begin{equation*}
\sum_{j} K_j K_j^{\dag} = \mathbb{I}.
\end{equation*}

##### Some single-qubit errors

Bit-flip error

    - The state of the qubit is chaged from |0⟩ to |1⟩ or vice-versa
    - key-operator is Pauli X
    - Kraus reprenetation 

\begin{equation*}
    \rho = (1-p) \rho + p X\rho X 
\end{equation*}
    with p in [0,1].


Phase-flip error

    - The relative phase of a qubit is changed, but it's magnitude remains the same
    - |0⟩ to -|0⟩ and |1⟩ to -|1⟩
    - key-operator is Pauli Z

\begin{equation*}
    \rho = (1-p) \rho + p Z\rho Z 
\end{equation*}
    with p in [0,1].    

Amplitude damping 

    - the qubit decayz from |1⟩ to the lower energy state |0⟩
    
\begin{equation*}
    \rho = K_1 \rho K_1^{\dag} + K_2 \rho K_2^{\dag}, 
\end{equation*}
 where $K_1 = [1,0;0,\sqrt{1-p}]$, $K_2 = [0,\sqrt{p};0,0]$ and p is the probability of decay.

#### 4.2 Noise modeling in CUDA Quantum with density-matrix simulator

    Bit-flip channel

In [2]:
import cudaq

# Set the target to our density matrix simulator.
cudaq.set_target('density-matrix-cpu')

# CUDA Quantum supports several different models of noise. In this case,
# we will examine the modeling of decoherence of the qubit state. This
# will occur from "bit flip" errors, wherein the qubit has a user-specified
# probability of undergoing an X-180 rotation.

# We will begin by defining an empty noise model that we will add
# these decoherence channels to.
noise = cudaq.NoiseModel()

# Bit flip channel with `1.0` probability of the qubit flipping 180 degrees.
bit_flip = cudaq.BitFlipChannel(1.0)
# We will apply this channel to any X gate on the qubit, giving each X-gate
# a probability of `1.0` of undergoing an extra X-gate.
noise.add_channel('x', [0], bit_flip)

# Now we may define our simple kernel function and allocate a register
# of qubits to it.
kernel = cudaq.make_kernel()
qubit = kernel.qalloc()

# Apply an X-gate to the qubit.
# It will remain in the |1> state with a probability of `1 - p = 0.0`.
kernel.x(qubit)
# Measure.
kernel.mz(qubit)

# Now we're ready to run the noisy simulation of our kernel.
# Note: We must pass the noise model to sample via key-word.
noisy_result = cudaq.sample(kernel, noise_model=noise)
noisy_result.dump()

# Our results should show all measurements in the |0> state, indicating
# that the noise has successfully impacted the system.

# To confirm this, we can run the simulation again without noise.
# We should now see the qubit in the |1> state.
noiseless_result = cudaq.sample(kernel)
noiseless_result.dump()

{ 0:1000 }
{ 1:1000 }


 Custom Noise Model

     Here, we demonstrate a custom noise model with the same Kraus operators as in the ampltiude damping channel, but following the same template we can build other noise models such as the Pauli noise model.

In [3]:
import cudaq
import numpy as np

# Set the target to our density matrix simulator.
cudaq.set_target('density-matrix-cpu')

# CUDA Quantum supports custom noise models through the definition of
# `KrausChannel`'s. In this case, we will define a set of `KrausOperator`'s
# that  affect the same noise as the `AmplitudeDampingChannel`. This
# channel will model the energy dissipation within our system via
# environmental interactions. With a variable probability, it will
# return the qubit to the |0> state.

# We will begin by defining an empty noise model that we will add
# our Kraus Channel to.
noise = cudaq.NoiseModel()


# We will define our Kraus Operators within functions, as to
# allow for easy control over the noise probability.
def kraus_operators(probability):
    """See Nielsen, Chuang Chapter 8.3.5 for definition source."""
    kraus_0 = np.array([[1, 0], [0, np.sqrt(1 - probability)]],
                       dtype=np.complex128)
    kraus_1 = np.array([[0, 0], [np.sqrt(probability), 0]], dtype=np.complex128)
    return [kraus_0, kraus_1]


# Manually defined amplitude damping channel with `1.0` probability
# of the qubit decaying to the ground state.
amplitude_damping = cudaq.KrausChannel(kraus_operators(1.0))
# We will apply this channel to any Hadamard gate on the qubit.
# Meaning, after each Hadamard on the qubit, there will be a
# probability of `1.0` that the qubit decays back to ground.
noise.add_channel('h', [0], amplitude_damping)

# Now we may define our simple kernel function and allocate a qubit.
kernel = cudaq.make_kernel()
qubit = kernel.qalloc()

# Then we apply a Hadamard gate to the qubit.
# This will bring it to `1/sqrt(2) (|0> + |1>)`, where it will remain
# with a probability of `1 - p = 0.0`.
kernel.h(qubit)

# Measure.
kernel.mz(qubit)

# Now we're ready to run the noisy simulation of our kernel.
# Note: We must pass the noise model to sample via key-word.
noisy_result = cudaq.sample(kernel, noise_model=noise)
noisy_result.dump()

# Our results should show all measurements in the |0> state, indicating
# that the noise has successfully impacted the system.

# To confirm this, we can run the simulation again without noise.
# The qubit will now have a 50/50 mix of measurements between
# |0> and |1>.
noiseless_result = cudaq.sample(kernel)
noiseless_result.dump()

{ 0:1000 }
{ 0:495 1:505 }


#### 3.2.2 Density matrix simulator
    * Simulates quantum circuits under the influence of noise. 
    * Currently, it calls the QPP library under the hood and has only CPU support.